In [1]:
from sage.structure.element import Element

# create a Multiplication Gate object with left, right, out, and an operation

class Gate(Element):
    '''(l) and (r) are lists of left or right input wires. Addition is compressed  (o) is
       a list containing the single output wire.'''
    def __init__(self, l, r, o, root):
        self.left = l
        self.right = r
        self.out = o
        self.root = root
        
    def __repr__(self):
        return ("Gate(" + str(self.left) + ", " + str(self.right) + ", " + self.out + ")")
    
# Parameters for BLS12-381, a pairing-friendly curve used in ZCash Sapling    
F=GF(0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab)
E=EllipticCurve(F, [0,4])

In [19]:

# circuit structure is defined with `gates` below
# this circuit is (c_1+c_2)*(c_3*c_4)=c_6
# example assignment: 2, 7, -3, 5, -135

n = 4 # inputs
s = 2 # multiplication gates
N = n + s # total number of wires

# note that this polynomial ring is over integers rather than a finite field due to limitations with sage

P.<x> = PolynomialRing(ZZ)

var('c_1 c_2 c_3 c_4 c_5 c_6')

wires = [c_1, c_2, c_3, c_4, c_5, c_6]

gates = [
    Gate([c_3], [c_4], [c_5], 0),
    Gate([c_1, c_2], [c_5], [c_6], 1)
]

roots = [g.root for g in gates]

In [28]:
# target polynomial
T = prod([x - g.root for g in gates])

# left polynomial
L = sum([c*prod([x - g.root + (1 if c in g.left else 0) for g in gates]) for c in wires])

# right polynomial
R = sum([c*prod([x - g.root + (1 if c in g.right else 0) for g in gates]) for c in wires])

# out polynomial
O = sum([c*prod([(x - g.root)^2 + (1 if c in g.out else 0) for g in gates]) for c in wires])

Q = L*R-O

print(L)

c_1*x^2 + c_2*x^2 + (x^2 - 1)*c_3 + (x^2 - x)*c_4 + (x^2 - x)*c_5 + (x^2 - x)*c_6


In [65]:
# prover knows avalid assignment: 2, 7, -3, 5, -135
# this includes intermediate step with -3 * 5 = -15 (c_5)

# program polynomial evaluated at this assignment

Qe1 = Q(2,7,-3,5,-15,-135)
print([Qe1(r) for r in roots])

# output of [0, 0] means Qe1 has roots at both roots of T, so both multiplication gates are correctly computed

[0, 0]


In [67]:
# here is an incorrect assignment

Qe2 = Q(2,7,-3,5,-15,-136)
print([Qe2(r) for r in roots])

# output of [0, 1] means Qe2 does not have a root for the second gate, so the second gate is incorrectly computed

[0, 1]
